In [1]:
from database.adatabase import ADatabase
from extractor.alp_client_extractor import ALPClientExtractor
from extractor.tiingo_extractor import TiingoExtractor
from processor.processor import Processor as processor
from datetime import datetime, timedelta
from tqdm import tqdm 
from dotenv import load_dotenv
load_dotenv()
import os
import pandas as pd
from time import sleep
import matplotlib.pyplot as plt
import numpy as np


In [24]:
window = 100
extractor = ALPClientExtractor(os.getenv("APCAKEY"),os.getenv("APCASECRET"))
tickers = ["AAVE", "AVAX", "BAT", "BTC", "CRV", "DOGE", "DOT", "ETH", "GRT", "LINK", "LTC", "MKR", "SHIB", "SUSHI", "UNI", "XTZ", "YFI"]
tickers = [x +"/USD" for x in tickers]
prices = []
start = datetime.now() - timedelta(days=5)
end = datetime.now()
for ticker in tickers:
    response = extractor.crypto_interval(ticker,start)
    price = pd.DataFrame(response["bars"][ticker]).rename(columns={"c":"adjclose","t":"date"})[["date","adjclose"]]
    price["date"] = pd.to_datetime(price["date"])
    price = price.sort_values("date")
    price["rolling_return"] = ((price["adjclose"].rolling(window).mean() - price["adjclose"]) / price["adjclose"])
    price["bollinger"] = ((price["adjclose"].rolling(window).mean() - price["adjclose"].rolling(window).std() - price["adjclose"]) / price["adjclose"])
    price["return"] = price["adjclose"].pct_change(5)
    price["avg_return"] = price["return"].rolling(window).mean()
    price["coev"] = (price["adjclose"].rolling(window).mean() / price["adjclose"].rolling(window).std())
    price["ticker"]= ticker
    prices.append(price)

sim = pd.concat(prices).sort_values("date").dropna()

In [25]:
metrics = [
    "rolling_return",
    "bollinger",
    "return",
    "avg_return",
    "coev"
]

In [26]:
import copy
positions = []
analysis = []
date_range = list(sim["date"].unique())
base_position = {"date":date_range[0],"ticker":"","quantity":0,"adjclose":0,"buy_price":0,"buy_date":date_range[0]}
for metric in metrics:
    for ascending in [True,False]:
        cash = 100
        position = copy.deepcopy(base_position)
        for date in tqdm(date_range):
            try:
                position = copy.deepcopy(position)
                position["date"] = date
                today = sim[sim["date"]==date]
                if today.index.size >= 0:
                    opportunity = today.sort_values(metric,ascending=ascending).iloc[0]
                    if position["ticker"] != "":
                        position["adjclose"] = today[today["ticker"]==position["ticker"]]["adjclose"].iloc[0].item()
                        pnl_req = ((position["adjclose"] - position["buy_price"]) / position["buy_price"] >= 0.05)
                        loss_req = ((position["adjclose"] - position["buy_price"]) / position["buy_price"] <= -0.05)
                        hpr_req = (date - position["buy_date"]).days > float(1/24)
                        if pnl_req or loss_req:
                            try:
                                cash = position["adjclose"] * position["quantity"]
                                position = copy.deepcopy(base_position)
                                position["date"] = date
                                position["ticker"] = opportunity["ticker"]
                                position["quantity"] = cash * 0.996 / opportunity["adjclose"] 
                                position["adjclose"] = opportunity["adjclose"]
                                position["buy_price"] = opportunity["adjclose"]
                                position["buy_date"] = date
                            except Exception as e:
                                print(str(e))
                                continue
                    else:
                            position["date"] = date
                            position["ticker"] = opportunity["ticker"]
                            position["quantity"] = cash * 0.996 / opportunity["adjclose"] 
                            position["adjclose"] = opportunity["adjclose"]
                            position["buy_price"] = opportunity["adjclose"]
                            position["buy_date"] = date
            except Exception as e:
                print(str(e))
        position["metric"] = metric
        position["ascending"] = ascending
        position["window"] = window
        position["pv"] = position["adjclose"] * position["quantity"]
        analysis.append(copy.deepcopy(position))

  3%|█████                                                                                                                                                                                               | 220/8600 [00:00<00:07, 1101.52it/s]

single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single posi

  5%|██████████                                                                                                                                                                                          | 442/8600 [00:00<00:07, 1107.04it/s]

single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single posi

  8%|███████████████▏                                                                                                                                                                                    | 664/8600 [00:00<00:07, 1099.64it/s]

...

single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single posi

 96%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍       | 8268/8600 [00:07<00:00, 1112.96it/s]

single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single posi

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8600/8600 [00:07<00:00, 1106.97it/s]

single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single posi

In [27]:
a = pd.DataFrame(analysis)
a.sort_values("pv",ascending=False)

,date,ticker,quantity,adjclose,buy_price,buy_date,metric,ascending,window,pv
7,2024-09-27 01:09:00+00:00,SHIB/USD,5.874747e+06,0.000019,0.000019,2024-09-27 01:03:00+00:00,avg_return,False,100,113.764477
5,2024-09-27 01:09:00+00:00,YFI/USD,2.053002e-02,5332.665000,5313.540000,2024-09-26 11:01:00+00:00,return,False,100,109.479692
4,2024-09-27 01:09:00+00:00,DOGE/USD,9.144439e+02,0.119365,0.117962,2024-09-26 16:48:00+00:00,return,True,100,109.152596
6,2024-09-27 01:09:00+00:00,BAT/USD,5.614377e+02,0.193889,0.194135,2024-09-26 14:16:00+00:00,avg_return,True,100,108.856880
9,2024-09-27 01:09:00+00:00,BTC/USD,1.647949e-03,65323.290500,63215.300000,2024-09-21 16:54:00+00:00,coev,False,100,107.649454
1,2024-09-27 01:09:00+00:00,AAVE/USD,6.483359e-01,166.030500,168.115000,2024-09-26 14:16:00+00:00,rolling_return,False,100,107.643537
3,2024-09-27 01:09:00+00:00,AAVE/USD,6.483359e-01,166.030500,168.115000,2024-09-26 14:16:00+00:00,bollinger,False,100,107.643537
2,2024-09-27 01:09:00+00:00,BAT/USD,5.518187e+02,0.193889,0.194698,2024-09-26 18:11:00+00:00,bollinger,True,100,106.991849
8,2024-09-27 01:09:00+00:00,LINK/USD,8.482099e+00,12.609950,12.180000,2024-09-24 22:51:00+00:00,coev,True,100,106.958846
0,2024-09-27 01:09:00+00:00,BTC/USD,1.566729e-03,65323.290500,62901.037500,2024-09-24 14:35:00+00:00,rolling_return,True,100,102.343888
